In [62]:
# #the preprocessing pipeline process serially:
# #take data into lists

# #data cleaning:
# convert to lower case
# remove stop words

# #preprocessing start:
# stemming
# tokenization
# vectorization

In [63]:
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [64]:
#take data into lists

with open("intents.json", 'r') as f:
    datastore = json.load(f)

tags = []
patterns = []
responses = []
unique_tags = []  #categorical data numeric form e ana

for item in datastore: #each {} in json file is an item. {} reprensts dictionary in python
    for sentence in item['patterns'] :
        patterns.append(sentence)
        tags.append(item['tag'])
    for ans in item['responses'] :    
        responses.append(ans)

# print(patterns)
# print(tags)
# print(responses)

In [ ]:
# unique category to number make dictionary like food classify
# #categorical data numeric form e ana

for item in datastore: #each {} in json file is an item
    unique_tags.append(item['tag'])
##print(unique_tags)

labels = []
for i in range(len(unique_tags)):
    labels.append(i)
#print(labels)

unique_tags_dictionary = dict(zip(labels, unique_tags))
print(unique_tags_dictionary)

In [66]:
#dictionary upor base catergory to number presentation
tags_category_num = []
for tag in tags:
    for keys,values in unique_tags_dictionary.items():
        if(tag == values):
            tags_category_num.append(keys)
#print(tags_category_num)


In [67]:
#for data cleaning:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\R
[nltk_data]     ONE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#stop words remove. then stemmer. then tokenize

#print(patterns) #list of sentence = ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'good morning'

# tf supports basic string normalization (lowercasing + punctuation stripping) 
# no support for  removing stop words
# just do the standardization beforehand
def parse_text(text):
    #print(f'Input: {text}')

    text = re.sub("[^a-zA-Z]", ' ', text) #re.sub(pattern, replacer, main string)
    #print(f'Remove punctuation and numbers: {text}')

    text = text.lower().split() # = ['is', 'anyone', 'there?']
    #print(f'Lowercase and split: {text}')

    swords = set(stopwords.words("english"))
    text = [w for w in text if w not in swords]
    #print(f'Remove stop words: {text}')

    text = " ".join(text)
    #print(f'Final: {text}')

    return text

#stemmer
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stem(text):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(text)


patterns_list = []
for sentence in patterns:
  patterns_list.append([sentence])
  
#data cleaning
for sublist in patterns_list: # sublist = ['Thanks a lot!']  
    for i in range(len(sublist)): #sublist[i] = Thanks a lot! #all sublist has one element the string. len(sublist)=1 for all
        sublist[i] = parse_text(sublist[i])

#print(patterns_list)

#stemming
for sublist in patterns_list: # sublist = ['Thanks a lot!']  
    for i in range(len(sublist)): #sublist[i] = Thanks a lot! #all sublist has one element the string. len(sublist)=1 for all
        sublist[i] = stem(sublist[i])


#'list of lists' theke 'list of string' e ana for vectorization.
patterns_new_list = []
for sublist in patterns_list: # sublist = ['Thanks a lot!']  
    for i in range(len(sublist)): #sublist[i] = 'Thanks a lot!' #all sublist has one element the string. len(sublist)=1 for all
        patterns_new_list.append(sublist[i])
print(patterns_new_list)
#print(patterns_list)

In [ ]:
#choosing 20% to be test data.
from sklearn.model_selection import train_test_split
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(patterns_new_list, tags_category_num, test_size= 0.2)

print(training_sentences)
print(training_labels)
# print(testing_sentences)
# print(testing_labels)

In [73]:
#vectorization
# alt is sklearn's CountVectorizer(ngram)
vocab_size = 10000
oov_tok = "<OOV>" #<OOV> is out of vocab token

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
#bug here

word_index = tokenizer.word_index
print(word_index)

max_length = 20 #our vector size is not too big
trunc_type='post'
padding_type='post'
#print(training_sequences)
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

{'<OOV>': 1, 'office': 2, 'offic': 3, 'attendance': 4, 'alchemy': 5, 'attend': 6, 'app': 7, 'provide': 8, 'outside': 9, 'mobile': 10, 'giving': 11, 'fe': 12, 'allow': 13, 'give': 14, 'icms': 15, 'exit': 16, 'day': 17, 'offered': 18, 'outdoor': 19, 'holiday': 20, 'take': 21, 'procedure': 22, 'alchemi': 23, 'use': 24, 'provided': 25, 'rule': 26, 'leav': 27, 'icm': 28, 'leave': 29, 'rul': 30, 'properti': 31, 'tim': 32, 'know': 33, 'govt': 34, 'dress': 35, 'rules': 36, 'put': 37, 'offer': 38, 'working': 39, 'polici': 40, 'month': 41, 'travel': 42, 'capture': 43, 'violation': 44, 'transportation': 45, 'privacy': 46, 'tour': 47, 'allowance': 48, 'work': 49, 'go': 50, 'phon': 51, 'server': 52, 'problem': 53, 'maximum': 54, 'suggestion': 55, 'violate': 56, 'bill': 57, 'time': 58, 'bil': 59, 'maintain': 60, 'cod': 61, 'communication': 62, 'mann': 63, 'indoor': 64, 'week': 65, 'photo': 66, 'hour': 67, 'daily': 68, 'log': 69, 'like': 70, 'viol': 71, 'types': 72, 'tell': 73, 'follow': 74, 'mandato

In [74]:
# Need this block to get it to work with TensorFlow 2.x NN
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

print(training_padded)
# print(training_labels)
# print(training_padded.shape)
# print(training_labels.shape)